In [1]:
import numpy as np
import pickle
import os

# 1. Load toàn bộ tập train
train_data_path  = "./train_data.npy"
train_labels_path = "./train_labels.pkl"

data = np.load(train_data_path)            # dtype: float32, shape: (N, T, P, J, C) ?
with open(train_labels_path, "rb") as f:
    labels = pickle.load(f)

print("Train data shape:", data.shape)
# Ví dụ: (1600, 30, 3, 17, 3)

print("Train labels shape:", labels.shape)
# Ví dụ: (1600,)

# 2. Kiểm tra sample đơn lẻ (để chắc chắn thứ tự axes)
sample = data[0]
print("\nSample[0] shape:", sample.shape)
# => (T, P, J, C)

# 3. In thử một vài giá trị min/max để xem scale
print("Sample[0] min, max per channel:", sample[...,0].min(), sample[...,0].max(),
      sample[...,1].min(), sample[...,1].max(),
      sample[...,2].min(), sample[...,2].max())

# 4. Nếu thấy axes sai (ví dụ shape=(T, P, C, J) hoặc (T, J, P, C)), hãy thử hoán permute
#    và xem shape mới:
# permuted = sample.transpose(0, 2, 3, 1)
# print("After transpose(0,2,3,1):", permuted.shape)


Train data shape: (1600, 16, 5, 17, 3)
Train labels shape: (1600,)

Sample[0] shape: (16, 5, 17, 3)
Sample[0] min, max per channel: 0.0 0.9567818 0.0 1.0 0.0 0.99173987


In [8]:
import torch

matrix_random = torch.rand(8, 3, 256, 256)
b, c, h, w = matrix_random.shape
pw, ph = 16, 16
nh = h // ph
nw = w // pw
matrix_random = torch.reshape(matrix_random, (b, c, nh, ph, nw, pw))

In [9]:
matrix_random.shape

torch.Size([8, 3, 16, 16, 16, 16])

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ffn_hidden_dim):
        super().__init__()
        # Self-Attention (SA)
        # kdim và vdim không cần thiết ở đây vì Query, Key, Value đều cùng chiều
        self.self_attention = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)
        
        # Cross-Attention (CA)
        # embed_dim là chiều đầu ra của CA (chiều của Query)
        # kdim và vdim là chiều của Key và Value (chiều của image_vector sau khi chiếu nếu cần)
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=embed_dim, 
            num_heads=num_heads,
            kdim=embed_dim, # image_vector sẽ được chiếu về embed_dim
            vdim=embed_dim, # image_vector sẽ được chiếu về embed_dim
            batch_first=True
        )
        self.norm2 = nn.LayerNorm(embed_dim)

        # Feed-Forward Network (FFN)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, ffn_hidden_dim),
            nn.GELU(), 
            nn.Linear(ffn_hidden_dim, embed_dim)
        )
        self.norm3 = nn.LayerNorm(embed_dim)

    def forward(self, text_features, image_vector):
        # text_features: (batch_size, seq_len, embed_dim)
        # image_vector: (batch_size, embed_dim) - đã được chiếu/xử lý để khớp chiều

        # 1. Self-Attention (SA)
        # Q, K, V đều là text_features
        sa_output, _ = self.self_attention(
            query=self.norm1(text_features), 
            key=self.norm1(text_features), 
            value=self.norm1(text_features)
        )
        text_features = text_features + sa_output 
        # 

        # 2. Cross-Attention (CA)
        # Q = text_features (từ văn bản)
        # K, V = image_vector (từ ảnh, cần mở rộng chiều)
        # image_vector.unsqueeze(1) biến (batch_size, embed_dim) thành (batch_size, 1, embed_dim)
        ca_output, _ = self.cross_attention(
            query=self.norm2(text_features), 
            key=self.norm2(image_vector.unsqueeze(1)), 
            value=self.norm2(image_vector.unsqueeze(1))
        )
        text_features = text_features + ca_output 
        # 

        # 3. Feed-Forward Network (FFN)
        ffn_output = self.feed_forward(self.norm3(text_features))
        text_features = text_features + ffn_output
        
        return text_features

class ImageGroundedTextEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ffn_hidden_dim, num_layers, img_hidden_dim):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Token đặc biệt [Encode]
        # Giả định ID của token [Encode] là 0 trong token_ids tổng quát
        self.encode_token_id = 0 
        self.encode_token_embedding = nn.Embedding(1, embed_dim) # Một embedding riêng cho token này

        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, ffn_hidden_dim)
            for _ in range(num_layers)
        ])
        
        # Lớp chiếu để đảm bảo chiều của image_vector khớp với embed_dim của text
        if img_hidden_dim != embed_dim:
            self.image_proj = nn.Linear(img_hidden_dim, embed_dim)
        else:
            self.image_proj = nn.Identity() # Nếu chiều đã khớp thì không làm gì

    def forward(self, text_token_ids, image_vector):
        # text_token_ids: (batch_size, seq_len - 1) - không bao gồm token [Encode]
        # image_vector: (batch_size, img_hidden_dim)

        # 1. Nhúng các token văn bản
        text_embeddings = self.token_embedding(text_token_ids)
        
        # 2. Tạo và thêm embedding của token [Encode] vào cuối chuỗi
        batch_size = text_token_ids.size(0)
        # Tạo tensor ID cho token [Encode] và nhúng nó
        encode_token_id_tensor = torch.tensor([self.encode_token_id], device=text_token_ids.device)
        encode_embedding = self.encode_token_embedding(encode_token_id_tensor).unsqueeze(0).repeat(batch_size, 1, 1)
        
        # Nối embedding của văn bản và token [Encode]
        combined_embeddings = torch.cat((text_embeddings, encode_embedding), dim=1) 

        # 3. Xử lý vector ảnh để khớp chiều (nếu cần)
        processed_image_vector = self.image_proj(image_vector)

        # 4. Chạy qua các lớp Transformer
        text_features = combined_embeddings
        for block in self.transformer_blocks:
            text_features = block(text_features, processed_image_vector)
        
        # 5. Lấy embedding của token [Encode] làm biểu diễn đa phương thức
        multimodal_representation = text_features[:, -1, :] 

        return multimodal_representation

# --- Ví dụ sử dụng và kiểm tra ---
if __name__ == "__main__":
    vocab_size = 10000     
    embed_dim = 256        
    num_heads = 8          
    ffn_hidden_dim = 512   
    num_layers = 2         
    img_hidden_dim = 768   

    model = ImageGroundedTextEncoder(vocab_size, embed_dim, num_heads, ffn_hidden_dim, num_layers, img_hidden_dim)
    
    batch_size = 4
    seq_len = 20 

    text_token_ids = torch.randint(1, vocab_size, (batch_size, seq_len)) 
    image_vector = torch.randn(batch_size, img_hidden_dim) 

    print(f"Kích thước đầu vào văn bản (text_token_ids): {text_token_ids.shape}")
    print(f"Kích thước đầu vào hình ảnh (image_vector): {image_vector.shape}")

    multimodal_output = model(text_token_ids, image_vector)

    print(f"Kích thước đầu ra biểu diễn đa phương thức: {multimodal_output.shape}") 
    # Mong đợi: (batch_size, embed_dim), tức là (4, 256)

Kích thước đầu vào văn bản (text_token_ids): torch.Size([4, 20])
Kích thước đầu vào hình ảnh (image_vector): torch.Size([4, 768])
Kích thước đầu ra biểu diễn đa phương thức: torch.Size([4, 256])


In [ ]:
import torch
import torch.nn as nn
import math

# --- Giả lập Positional Encoding cho đơn giản ---
# Trong thực tế, bạn sẽ dùng một implement phức tạp hơn.
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x có shape [seq_len, batch_size, embedding_dim]
        x = x + self.pe[:x.size(0)]
        return x

# --- Khối Decoder cơ bản (Module 3) ---
# Mỗi khối này chứa: Causal Self-Attention -> Cross-Attention -> Feed Forward
class ImageGroundedDecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # 1. Lớp Causal Self-Attention
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        
        # 2. Lớp Cross-Attention
        self.cross_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        
        # 3. Feed Forward Network
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, dim_feedforward),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim_feedforward, embed_dim)
        )

        # Các lớp chuẩn hóa và dropout
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt, memory, tgt_mask=None, is_causal=False):
        """
        Args:
            tgt (Tensor): Chuỗi văn bản đầu vào. Shape: [batch_size, tgt_len, embed_dim]
            memory (Tensor): Vector đặc trưng của ảnh. Shape: [batch_size, 1, embed_dim]
            tgt_mask (Tensor): Mask cho self-attention.
            is_causal (bool): Cờ để bật/tắt Causal Attention tự động.
        """
        # --- Bước 1: Causal Self-Attention ---
        # Query, Key, Value đều là `tgt`.
        # Sử dụng cờ is_causal=True để Pytorch tự tạo mask.
        attn_output, _ = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask, is_causal=is_causal)
        # Residual connection và Layer Norm
        tgt = self.norm1(tgt + self.dropout(attn_output))

        # --- Bước 2: Cross-Attention ---
        # Query là `tgt` (kết quả từ bước trên), Key và Value là `memory` (từ ảnh).
        # Không cần mask ở đây.
        attn_output, _ = self.cross_attn(query=tgt, key=memory, value=memory)
        # Residual connection và Layer Norm
        tgt = self.norm2(tgt + self.dropout(attn_output))

        # --- Bước 3: Feed Forward Network ---
        ffn_output = self.ffn(tgt)
        # Residual connection và Layer Norm
        tgt = self.norm3(tgt + self.dropout(ffn_output))
        
        return tgt

# --- Toàn bộ mô hình Decoder ---
class ImageGroundedDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, dim_feedforward):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        # Bỏ qua PositionalEncoding đơn giản nếu không cần thiết cho ví dụ
        # self.pos_encoder = PositionalEncoding(embed_dim)
        self.embed_dim = embed_dim
        
        # Tạo một chuỗi các Decoder Blocks
        self.layers = nn.ModuleList([
            ImageGroundedDecoderBlock(embed_dim, num_heads, dim_feedforward) 
            for _ in range(num_layers)
        ])
        
        # Lớp Linear cuối cùng để map ra không gian từ vựng
        self.fc_out = nn.Linear(embed_dim, vocab_size)

    def forward(self, image_features, tgt_tokens):
        """
        Args:
            image_features (Tensor): Đặc trưng ảnh. Shape [batch_size, embed_dim]
            tgt_tokens (Tensor): Chuỗi token văn bản. Shape [batch_size, seq_len]
        """
        # 1. Embedding và Positional Encoding cho văn bản
        # Chuyển shape thành [batch_size, seq_len, embed_dim]
        tgt_embed = self.token_embedding(tgt_tokens) * math.sqrt(self.embed_dim)
        # tgt_embed = self.pos_encoder(tgt_embed.permute(1,0,2)).permute(1,0,2) # Nếu dùng PositionalEncoding
        
        # 2. Định dạng lại vector ảnh để phù hợp với MultiheadAttention
        # `memory` cần có shape [batch_size, num_img_tokens, embed_dim].
        # Vì ta giả định ảnh chỉ là 1 vector, nên num_img_tokens = 1.
        memory = image_features.unsqueeze(1) # Shape: [batch_size, 1, embed_dim]
        
        # 3. Đưa qua các lớp DecoderBlock
        # Bật cờ is_causal trong forward pass của block đầu tiên hoặc của tất cả
        # để đảm bảo tính nhân quả.
        output = tgt_embed
        for layer in self.layers:
            output = layer(output, memory, is_causal=True)
            
        # 4. Đưa qua lớp Linear cuối để có được logits
        logits = self.fc_out(output)
        
        return logits

# --- Ví dụ sử dụng ---
if __name__ == '__main__':
    # Hyperparameters
    BATCH_SIZE = 4
    SEQ_LENGTH = 15  # Độ dài chuỗi caption
    VOCAB_SIZE = 1000 # Kích thước từ vựng
    EMBED_DIM = 512  # Kích thước embedding (phải chia hết cho num_heads)
    NUM_HEADS = 8    # Số lượng attention heads
    NUM_LAYERS = 6   # Số lớp decoder
    DIM_FFN = 2048   # Chiều của lớp ẩn trong FFN

    # Khởi tạo model
    decoder = ImageGroundedDecoder(
        vocab_size=VOCAB_SIZE,
        embed_dim=EMBED_DIM,
        num_heads=NUM_HEADS,
        num_layers=NUM_LAYERS,
        dim_feedforward=DIM_FFN
    )
    print(decoder)

    # --- Tạo dữ liệu giả lập ---
    # 1. Vector đặc trưng của ảnh (đầu ra của ViT đã được đơn giản hóa)
    # Shape: [batch_size, embed_dim]
    anh_feature = torch.randn(BATCH_SIZE, EMBED_DIM)

    # 2. Chuỗi văn bản đầu vào cho decoder (ví dụ: "A cat sitting on...")
    # Thường bắt đầu bằng token [Decode] hoặc [SOS]
    # Shape: [batch_size, seq_len]
    chuoi_van_ban = torch.randint(0, VOCAB_SIZE, (BATCH_SIZE, SEQ_LENGTH))

    # --- Forward pass ---
    logits = decoder(anh_feature, chuoi_van_ban)

    print(f"\nShape của vector ảnh đầu vào: {anh_feature.shape}")
    print(f"Shape của chuỗi văn bản đầu vào: {chuoi_van_ban.shape}")
    print(f"Shape của logits đầu ra: {logits.shape}") # Mong đợi [batch_size, seq_len, vocab_size]

In [ ]:
import torch
import torch.nn as nn
import math

# ===================================================================
# ===== CÁC MODULE CƠ BẢN VÀ THÀNH PHẦN GIẢ LẬP =====================
# ===================================================================

class SimpleImageEncoder(nn.Module):
    """
    Module giả lập cho Visual Transformer (ViT).
    Nó nhận một "ảnh" giả và biến nó thành một chuỗi các vector đặc trưng.
    """
    def __init__(self, image_size=224, patch_size=32, in_channels=3, embed_dim=512):
        super().__init__()
        num_patches = (image_size // patch_size) ** 2
        self.patch_embedding = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.positional_embedding = nn.Parameter(torch.randn(1, num_patches + 1, embed_dim))
        # Trong một ViT thực sự, sẽ có các lớp Transformer Encoder ở đây
        self.transformer_encoder = nn.Identity() # Bỏ qua để đơn giản hóa

    def forward(self, img):
        # img shape: [batch_size, 3, 224, 224]
        x = self.patch_embedding(img).flatten(2).transpose(1, 2) # Shape: [batch_size, num_patches, embed_dim]
        
        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1) # Shape: [batch_size, num_patches + 1, embed_dim]
        
        x += self.positional_embedding
        x = self.transformer_encoder(x)
        return x # Trả về chuỗi các patch embedding, bao gồm cả [CLS] token

class PositionalEncoding(nn.Module):
    # Lớp Positional Encoding chuẩn
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x có shape [batch_size, seq_len, embedding_dim]
        x = x + self.pe[:, :x.size(1)]
        return x

# ===================================================================
# ===== MODULE 1: UNIMODAL ENCODER ==================================
# ===================================================================

class UnimodalTextEncoder(nn.Module):
    """Mã hóa văn bản một cách độc lập, tương tự như BERT."""
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, dim_feedforward, dropout=0.1):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoder = PositionalEncoding(embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, 
            nhead=num_heads, 
            dim_feedforward=dim_feedforward, 
            dropout=dropout,
            batch_first=True # Rất quan trọng!
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.embed_dim = embed_dim

    def forward(self, text_tokens):
        # text_tokens shape: [batch_size, seq_len]
        x = self.token_embedding(text_tokens) * math.sqrt(self.embed_dim)
        x = self.pos_encoder(x)
        x = self.dropout(x)
        encoded_text = self.transformer_encoder(x)
        
        # Lấy vector của [CLS] token (giả sử nó luôn ở vị trí 0)
        cls_output = encoded_text[:, 0, :]
        
        return encoded_text, cls_output

# ===================================================================
# ===== MODULE 2: IMAGE-GROUNDED TEXT ENCODER =======================
# ===================================================================

class GroundedEncoderBlock(nn.Module):
    """Block cho Module 2: Self-Attention -> Cross-Attention -> FFN"""
    def __init__(self, embed_dim, num_heads, dim_feedforward, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, dim_feedforward), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(dim_feedforward, embed_dim)
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, txt, img_memory):
        # 1. Self-Attention (văn bản tự chú ý đến chính nó)
        txt2, _ = self.self_attn(txt, txt, txt)
        txt = self.norm1(txt + self.dropout(txt2))
        
        # 2. Cross-Attention (văn bản chú ý đến ảnh)
        txt2, _ = self.cross_attn(query=txt, key=img_memory, value=img_memory)
        txt = self.norm2(txt + self.dropout(txt2))
        
        # 3. FFN
        txt2 = self.ffn(txt)
        txt = self.norm3(txt + self.dropout(txt2))
        return txt

class ImageGroundedTextEncoder(nn.Module):
    """Mã hóa văn bản có điều kiện là ảnh."""
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, dim_feedforward):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoder = PositionalEncoding(embed_dim)
        self.layers = nn.ModuleList([
            GroundedEncoderBlock(embed_dim, num_heads, dim_feedforward) for _ in range(num_layers)
        ])
        self.embed_dim = embed_dim

    def forward(self, text_tokens, image_features):
        # text_tokens shape: [batch_size, seq_len]
        # image_features shape: [batch_size, num_patches, embed_dim]
        txt_embed = self.token_embedding(text_tokens) * math.sqrt(self.embed_dim)
        txt_embed = self.pos_encoder(txt_embed)

        output = txt_embed
        for layer in self.layers:
            output = layer(output, image_features)
            
        # Lấy vector của [Encode] token (giả sử nó ở vị trí cuối cùng)
        encode_output = output[:, -1, :]
        return output, encode_output

# ===================================================================
# ===== MODULE 3: IMAGE-GROUNDED TEXT DECODER =======================
# ===================================================================

class GroundedDecoderBlock(nn.Module):
    """Block cho Module 3: Causal Self-Attention -> Cross-Attention -> FFN"""
    def __init__(self, embed_dim, num_heads, dim_feedforward, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, dim_feedforward), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(dim_feedforward, embed_dim)
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt, img_memory):
        # 1. Causal Self-Attention (dùng cờ is_causal)
        tgt2, _ = self.self_attn(tgt, tgt, tgt, is_causal=True)
        tgt = self.norm1(tgt + self.dropout(tgt2))
        
        # 2. Cross-Attention (văn bản chú ý đến ảnh)
        tgt2, _ = self.cross_attn(query=tgt, key=img_memory, value=img_memory)
        tgt = self.norm2(tgt + self.dropout(tgt2))
        
        # 3. FFN
        tgt2 = self.ffn(tgt)
        tgt = self.norm3(tgt + self.dropout(tgt2))
        return tgt

class ImageGroundedTextDecoder(nn.Module):
    """Sinh văn bản có điều kiện là ảnh."""
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, dim_feedforward):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoder = PositionalEncoding(embed_dim)
        self.layers = nn.ModuleList([
            GroundedDecoderBlock(embed_dim, num_heads, dim_feedforward) for _ in range(num_layers)
        ])
        self.fc_out = nn.Linear(embed_dim, vocab_size)
        self.embed_dim = embed_dim

    def forward(self, tgt_tokens, image_features):
        # tgt_tokens shape: [batch_size, seq_len]
        # image_features shape: [batch_size, num_patches, embed_dim]
        tgt_embed = self.token_embedding(tgt_tokens) * math.sqrt(self.embed_dim)
        tgt_embed = self.pos_encoder(tgt_embed)

        output = tgt_embed
        for layer in self.layers:
            output = layer(output, image_features)
            
        logits = self.fc_out(output)
        return logits

# ===================================================================
# ===== MÔ HÌNH HỢP NHẤT MED ========================================
# ===================================================================

class MED(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, dim_feedforward):
        super().__init__()
        
        # Khởi tạo các thành phần con
        self.image_encoder = SimpleImageEncoder(embed_dim=embed_dim)
        self.unimodal_text_encoder = UnimodalTextEncoder(vocab_size, embed_dim, num_heads, num_layers, dim_feedforward)
        self.grounded_encoder = ImageGroundedTextEncoder(vocab_size, embed_dim, num_heads, num_layers, dim_feedforward)
        self.decoder = ImageGroundedTextDecoder(vocab_size, embed_dim, num_heads, num_layers, dim_feedforward)
        
        # Chú ý: Trong một mô hình thực tế, bạn có thể chia sẻ trọng số,
        # ví dụ như self.token_embedding, giữa các module.
        # Ở đây, để rõ ràng, chúng được khởi tạo riêng biệt.

    def forward(self, mode, **kwargs):
        if mode == 'unimodal_image':
            return self.image_encoder(kwargs['image'])
        elif mode == 'unimodal_text':
            return self.unimodal_text_encoder(kwargs['text_tokens'])
        elif mode == 'grounded_encoder':
            image_features = self.image_encoder(kwargs['image'])
            return self.grounded_encoder(kwargs['text_tokens'], image_features)
        elif mode == 'decoder':
            image_features = self.image_encoder(kwargs['image'])
            return self.decoder(kwargs['text_tokens'], image_features)
        else:
            raise ValueError(f"Unknown mode: {mode}")

# ===================================================================
# ===== VÍ DỤ SỬ DỤNG ===============================================
# ===================================================================
if __name__ == '__main__':
    # Hyperparameters
    BATCH_SIZE = 4
    SEQ_LENGTH = 20
    VOCAB_SIZE = 1000
    EMBED_DIM = 512
    NUM_HEADS = 8
    NUM_LAYERS = 6
    DIM_FFN = 2048

    # Khởi tạo mô hình hợp nhất
    med_model = MED(
        vocab_size=VOCAB_SIZE,
        embed_dim=EMBED_DIM,
        num_heads=NUM_HEADS,
        num_layers=NUM_LAYERS,
        dim_feedforward=DIM_FFN
    )

    # --- Tạo dữ liệu giả lập ---
    dummy_image = torch.randn(BATCH_SIZE, 3, 224, 224)
    dummy_text = torch.randint(0, VOCAB_SIZE, (BATCH_SIZE, SEQ_LENGTH))

    print("================ CHẠY THỬ CÁC MODULE ================")

    # --- 1. Chế độ Unimodal ---
    print("\n--- 1. Chế độ Unimodal ---")
    img_features_unimodal = med_model(mode='unimodal_image', image=dummy_image)
    print(f"Image Encoder Output Shape: {img_features_unimodal.shape}")
    
    encoded_text_unimodal, cls_out = med_model(mode='unimodal_text', text_tokens=dummy_text)
    print(f"Text Encoder Output Shape: {encoded_text_unimodal.shape}")
    print(f"Text Encoder [CLS] Output Shape: {cls_out.shape}")

    # --- 2. Chế độ Image-grounded Encoder ---
    print("\n--- 2. Chế độ Image-grounded Encoder (Understanding) ---")
    # Giả sử token [Encode] được thêm vào cuối
    encoded_grounded, encode_out = med_model(mode='grounded_encoder', image=dummy_image, text_tokens=dummy_text)
    print(f"Grounded Encoder Output Shape: {encoded_grounded.shape}")
    print(f"Grounded Encoder [Encode] Output Shape: {encode_out.shape}")

    # --- 3. Chế độ Image-grounded Decoder ---
    print("\n--- 3. Chế độ Image-grounded Decoder (Generation) ---")
    # Giả sử chuỗi đầu vào cho decoder là `dummy_text`
    logits = med_model(mode='decoder', image=dummy_image, text_tokens=dummy_text)
    print(f"Decoder Logits Output Shape: {logits.shape}")